<a href="https://colab.research.google.com/github/mathgds/introducao_a_ciencia_de_dados/blob/main/03_data_split_and_cross_validation_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstração de divisão de dados e validação cruzada

## 1. Introdução
Este caderno oferece uma introdução à regressão linear. Vamos criar um conjunto de dados sintético, ajustar um modelo de regressão linear e avaliar seu desempenho em quatro cenários diferentes:

1. Sem dividir os dados
2. Com os dados divididos em conjuntos de treinamento e teste
3. Usando validação cruzada
4. Comparando diferentes números de divisões (folds) na validação cruzada

## 2. Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression # Importa o módulo LinearRegression da biblioteca scikit-learn,
                                                  # que fornece ferramentas para criar e treinar modelos de regressão linear.

from sklearn.model_selection import train_test_split, cross_val_score # 1. train_test_split: Usada para dividir um conjunto de dados em conjuntos de treinamento e teste.
                                                                      # 2. cross_val_score: Usada para realizar validação cruzada e avaliar o
                                                                      # desempenho do modelo em diferentes divisões do conjunto de dados.

from sklearn.metrics import mean_squared_error, r2_score # mean_squared_error: Calcula o Erro Quadrático Médio (MSE), que mede a média dos quadrados dos erros,
                                                         # ou seja, a diferença entre valores previstos e reais.
                                                         # r2_score: Calcula o coeficiente de determinação (R²), que indica a proporção da variância dos dados
                                                         # que é explicada pelo modelo.

## 3. Criando um conjunto de dados sintéticos
Definindo o tamanho do conjunto de dados.

In [ ]:
# Gerando dados aleatórios
n_samples = 100     # Define o número de amostras (ou linhas) no conjunto de dados como 100.

np.random.seed(42)  # Define uma semente para o gerador de números aleatórios, garantindo que os resultados sejam reproduzíveis.
                    # A mesma semente produz os mesmos números aleatórios em execuções diferentes.

X = 2 * np.random.rand(n_samples, 1)  # Gera uma matriz X com n_samples linhas e 1 coluna, preenchida com números aleatórios uniformemente distribuídos entre 0 e 2.

y = 4 + 3 * X + np.random.randn(n_samples, 1) # Cria o vetor y como uma função linear de X (com intercepto 4 e coeficiente 3) e
                                              # adiciona um ruído aleatório (distribuição normal com média 0 e desvio padrão 1) para simular dados reais.

# Criando um DataFrame
data = pd.DataFrame(data=np.hstack((X, y)), columns=['Feature', 'Target'])

# Exibe as 5 primeiras linhas do DataFrame
data.head() # Exibe as primeiras 5 linhas do DataFrame para uma rápida visualização dos dados gerados.

## 4. Análise Exploratória de Dados
Visualizando a relação entre a característica e o alvo.

In [ ]:
# Criando o Gráfico de Dispersão
plt.scatter(data['Feature'], data['Target'])

# Adicionando Rótulos aos Eixos
plt.xlabel('Feature')
plt.ylabel('Target')

# Adicionando um Título ao Gráfico
plt.title('Feature vs Target')

# Exibindo o Gráfico
plt.show()

## 5. Cenário 1: Sem divisão dos dados

In [ ]:
# Treinando o modelo de regressao linear
model = LinearRegression() # Cria uma instância do modelo de regressão linear da biblioteca scikit-learn.
model.fit(data[['Feature']], data['Target']) # Treina o modelo de regressão linear usando o método fit.

In [ ]:
# Avaliando o modelo
y_pred = model.predict(data[['Feature']]) # Usa o modelo treinado para fazer previsões com base nos dados da coluna 'Feature'.
                                          # O método predict gera previsões da variável dependente ('Target') para cada entrada na variável independente ('Feature').

mse1 = mean_squared_error(data['Target'], y_pred) # Calcula o Erro Quadrático Médio (MSE) entre os valores reais (data['Target']) e os valores previstos (y_pred).
r2_1 = r2_score(data['Target'], y_pred) # Calcula o coeficiente de determinação (R²) que indica a proporção da variância dos dados que é explicada pelo modelo.
print(f'Scenario 1 - MSE: {mse1}')
print(f'Scenario 1 - R²: {r2_1}')

## 6. Cenário 2: Dividindo os dados em conjuntos de treinamento e teste

In [ ]:
# Dividindo os dados
X_train, X_test, y_train, y_test = train_test_split(data[['Feature']], data['Target'], test_size=0.2, random_state=42)
# train_test_split: Esta função divide o conjunto de dados em dois conjuntos: treinamento e teste.

# O resultado da função train_test_split é atribuído a quatro variáveis:

# X_train: Dados de entrada para treinamento.
# X_test: Dados de entrada para teste.
# y_train: Rótulos (ou valores) para treinamento.
# y_test: Rótulos (ou valores) para teste.

# Treinando o modelo de regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Evaluating the Model on Training Data
y_train_pred = model.predict(X_train)

# Usa o modelo treinado para fazer previsões com base nos dados de treinamento (X_train).
# A saída, y_train_pred, contém as previsões do modelo para os valores de y correspondentes aos dados de entrada em X_train.

mse2_train = mean_squared_error(y_train, y_train_pred)  # Calcula o Erro Quadrático Médio (MSE) para os dados de treinamento.

r2_2_train = r2_score(y_train, y_train_pred)  # Calcula o coeficiente de determinação (R²) para os dados de treinamento.

print(f'Scenario 2 - Training MSE: {mse2_train}')
print(f'Scenario 2 - Training R²: {r2_2_train}')

In [ ]:
# Avaliando o modelo com dados de teste
y_test_pred = model.predict(X_test)
mse2_test = mean_squared_error(y_test, y_test_pred)
r2_2_test = r2_score(y_test, y_test_pred)
print(f'Scenario 2 - Testing MSE: {mse2_test}')
print(f'Scenario 2 - Testing R²: {r2_2_test}')

## 7. Cenário 3: Usando Validação Cruzada

In [ ]:
# Validação Cruzada

# cross_val_score: Esta função realiza a validação cruzada, que é uma técnica para avaliar o desempenho do modelo de forma mais robusta,
#                dividindo os dados em múltiplas partes e treinando/testando o modelo em cada parte.

cv_mse = -cross_val_score(model, data[['Feature']], data['Target'], cv=5, scoring='neg_mean_squared_error')
cv_r2 = cross_val_score(model, data[['Feature']], data['Target'], cv=5, scoring='r2')

# Avaliando o modelo
mse3 = cv_mse.mean()
r2_3 = cv_r2.mean()
print(f'Scenario 3 - MSE: {mse3}')
print(f'Scenario 3 - R²: {r2_3}')

## 8. Cenário 4: Comparando diferentes números de partições na Validação Cruzada

In [ ]:
# Comparing different numbers of folds in cross-validation

folds = [3, 5, 10, 20] # Definindo o numero de partições

# Listas vazias que serão usadas para armazenar as médias dos MSE e R² para cada número de folds.
mse_scores = []
r2_scores = []

for k in folds: # O loop for itera sobre cada valor de k na lista folds, onde k representa o número de folds para a validação cruzada.

    cv_mse = -cross_val_score(model, data[['Feature']], data['Target'], cv=k, scoring='neg_mean_squared_error')
    # cross_val_score: Realiza a validação cruzada com k folds.
    # scoring='neg_mean_squared_error': Calcula o MSE, retornado como valor negativo, por isso o - é usado para inverter o sinal e obter o valor positivo do MSE.

    cv_r2 = cross_val_score(model, data[['Feature']], data['Target'], cv=k, scoring='r2')
    # scoring='r2': Calcula o coeficiente de determinação (R²).

    mse_scores.append(cv_mse.mean())
    # Calcula a média dos MSEs obtidos durante a validação cruzada para o número atual de folds e adiciona à lista mse_scores.
    r2_scores.append(cv_r2.mean())
    # Calcula a média dos R² obtidos durante a validação cruzada para o número atual de folds e adiciona à lista r2_scores.

    print(f'{k}-fold CV - MSE: {cv_mse.mean()}')
    print(f'{k}-fold CV - R²: {cv_r2.mean()}')

In [ ]:
# Exibindo os resultados
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(folds, mse_scores, marker='o')
plt.title('MSE vs. Number of Folds')
plt.xlabel('Number of Folds')
plt.ylabel('MSE')

plt.subplot(1, 2, 2)
plt.plot(folds, r2_scores, marker='o')
plt.title('R² vs. Number of Folds')
plt.xlabel('Number of Folds')
plt.ylabel('R²')

plt.tight_layout()
plt.show()

## 9. Comparação de todos os cenários

In [ ]:
# Comparando todos os cenários
#scenarios = ['No Split', 'Train/Test Split (Train)', 'Train/Test Split (Test)', '5-Fold CV', '3-Fold CV', '10-Fold CV', '20-Fold CV']
#mse_all = [mse1, mse2_train, mse2_test, mse_cv] + mse_scores
#r2_all = [r2_1, r2_2_train, r2_2_test, r2_cv] + r2_scores

scenarios = ['No Split', 'Train/Test Split (Train)', 'Train/Test Split (Test)', '3-Fold CV', '5-Fold CV', '10-Fold CV', '20-Fold CV']
mse_all = [mse1, mse2_train, mse2_test] + mse_scores
r2_all = [r2_1, r2_2_train, r2_2_test] + r2_scores

# Plotting the comparison
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.barh(scenarios, mse_all, color='skyblue')
plt.xlabel('MSE')
plt.title('Comparison of MSE across Scenarios')

plt.subplot(1, 2, 2)
plt.barh(scenarios, r2_all, color='lightgreen')
plt.xlabel('R²')
plt.title('Comparison of R² across Scenarios')

plt.tight_layout()
plt.show()

## 10. Conclusão
Nesse caderno, nós criamos um conjunto de dados sintéticos e os ajustamos à uma regressão linear. Nós validamos a performance do modelo usando MSE e R² em quatro diferentes cenários:

1. Sem dividir os dados
2. Dividindo os dados em conjuntos de treinamento e teste
3. Utilizando Validação Cruzada
4. Comparando diferente números de partições na validação cruzada

Por fim, nós comparamos os resultados de todos os cenários para entender o impacto de diferentes métodos de validação e o número de partições no desempenho do modelo.
